# Revised case normalization for Hirslanden Zürich 2016

Notes:





In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group, format_for_grouper_one_case, format_for_grouper
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Zurich.xlsx'),
                     'Hirslanden Klinik Zurich', '2016', 'Änderungen_Hirslanden 2016')

print(file_info)

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Zurich.xlsx', hospital_name_db='Hirslanden Klinik Zurich', year='2016', sheet='Änderungen_Hirslanden 2016')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 09:11:26.492 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 104 cases for Hirslanden Klinik Zurich 2016
2022-11-21 09:11:26.497 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
datum                                                string
unnamed: 1                                           string
case_id                                              object
patient_id                                           object
gender                                               string
age_years                                             int64
bfs_code                                             string
duration_of_stay                                      int64
pflegetage neu                                       string
pccl                                                  int64
pccl neu                                             string
old_pd                                               string


,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,40822411,19CA20CD5E408064,M,86,12,40822411,C61,C61,M200,3,M09A,[F058],[],[],[]
1,40828212,010B949C1F25E5CA,M,73,11,40828212,D333,D333,M200,3,B20A,[E891],[],[],"[8703::20160104, 8703::20160108, 8703::20160111, 8703::20160112]"
2,40835992,8F5D8BEA91706191,M,57,28,40835992,M4787,M4787,M200,3,I09D,"[I982, N179]",[],[],[]
3,40836137,BF0089E1CBFF5DF6,M,75,5,40836137,N40,N40,M200,3,M02B,"[N183, E1172, N083, D860]",[],[],[]
4,40838008,AABE4CDA053DAE66,W,78,10,40838008,M161,M161,M200,3,I47Z,"[E611, I81]",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,41108786,AFCE26A3C48249DE,M,68,4,41108786,K210,K210,M100,2,G72B,[F102],[],[],[]
100,41145363,946C45A5B6534021,M,85,4,41145363,C61,C61,M200,3,M09B,[N183],[],[],[]
101,41165859,DABE6EC817B1FC84,M,82,12,41165859,M4807,M4807,M200,3,I09D,[R31],[],[],[]
102,41176991,391DC257DDCFD2BA,W,74,16,41176991,C240,C240,M100,3,H41B,[L8925],[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)

revised_cases.head()


2022-11-21 09:11:33.832 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 19282 rows from the DB, for the hospital 'Hirslanden Klinik Zurich' in 2016
2022-11-21 09:11:33.848 | WARNING  | src.revised_case_normalization.notebook_functions.revise:revise:51 - 11 rows could not be matched, given ['age_years', 'case_id_norm', 'duration_of_stay', 'gender']


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,336518,40944820,J4419,"[U6900, I2728, K703, I1090, R53, ...]",992212::20160325,"[992909::20160401, 874199::20160401, 880199::20160401, 890A32::20160328]",W,66,0,0,11,0,01,2016-03-25,0,00,2016-04-05
1,336577,40858393,J4481,"[J47, M8189, F328, I1090, J9600, ...]",874199::20160119,[],W,67,0,0,8,0,01,2016-01-15,0,00,2016-01-23
2,336604,40860914,C251,"[C772, I871, K660, N838, E441, ...]",525211::20160121,"[415::20160121, 009A13::20160121, 399199::20160121, 99B713::20160121, 8993::20160127, ...]",W,67,0,0,12,0,01,2016-01-20,0,00,2016-02-01
3,336680,41087350,C9000,"[M9072, Z923, G562, Y849, Z926, ...]",7782:R:20160830,"[845511:R:20160830, 785299:R:20160830, 044912:R:20160830]",W,53,0,0,9,0,01,2016-08-30,0,00,2016-09-08
4,337169,41112106,I214,"[I5013, E785, I1090, M7970, E890, ...]",006629::20160907,"[360799::20160907, 0040::20160907, 0045::20160907, 8853::20160907, 3722::20160907, ...]",W,75,0,0,4,0,01,2016-09-06,0,00,2016-09-10


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops,aimedic_id,case_id_db,patient_id_db,age_days,admission_weight,gestation_age,admission_date,grouper_admission_type,discharge_date,grouper_discharge_type,ventilation_hours
2,40835992,8F5D8BEA91706191,M,57,28,40835992,M4787,M4787,M200,3,I09D,"[I982, N179]",[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,40859453,15626465C742858E,W,83,16,40859453,I7024,I7024,M200,3,F14B,[N183],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,40866702,7EE3A5CB27720057,M,64,23,40866702,N328,N328,M200,3,L18B,"[T810, D6830]",[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,40923263,58A235C8E805AF88,M,76,22,40923263,M4806,M4806,M100,3,I09D,[I803],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,40981454,81A204C7F57B3960,W,72,16,40981454,I6710,I6710,M850,3,B20C,[I650],[],[],"[8841:L:20160531, 393019:B:20160531, 889110::20160601]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,41053612,E030E0D8DE0AF1BE,M,59,29,41053612,T828,T828,M200,3,F34B,[I743],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,41070240,DA73367B06968193,W,89,19,41070240,I352,I352,M100,3,F49C,"[I1100, I5012]",[I1090],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,41077147,F10E46209BD8A722,M,64,20,41077147,M8448,M8448,M200,3,I10C,"[D6830, E876]",[],[990510],"[030991::20160805, 030934::20160805]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,40996051,3842CF40EC7C1250,M,84,28,40996051,M4806,M4806,M200,3,I09D,[N183],[N182],[],"[890809::20160519, 889714::20160606]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,41019204,7D8346167A78D381,W,64,11,41019204,N390,N390,M200,3,L18B,[D685],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# The row 43 can not be grouped, we deleted it at the moment
revised_cases.drop(43, inplace=True)
revised_cases.shape

(92, 17)

In [7]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 09:11:34.553 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 92 cases ...
2022-11-21 09:11:35.618 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 92 cases into: 92 revisions, 877 diagnoses rows, 568 procedure rows


In [8]:
procedures_update

,aimedic_id,code,side,date,is_grouper_relevant,is_primary
0,336518,992212,,2016-03-25,False,False
1,336518,992909,,2016-04-01,False,False
2,336518,874199,,2016-04-01,False,False
3,336518,880199,,2016-04-01,False,False
4,336518,890A32,,2016-03-28,False,False
...,...,...,...,...,...,...
563,355135,815111,L,2016-07-26,True,False
564,355135,009940,,2016-07-26,False,False
565,355135,874199,,2016-07-29,False,False
566,355135,883850,,2016-07-29,False,False


In [9]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,336518,E65A,1.16,1.16,4,2022-12-31
1,336577,E65A,1.16,1.16,4,2022-12-31
2,336604,H09A,3.30,3.30,4,2022-12-31
3,336680,R02Z,4.26,4.26,4,2022-12-31
4,337169,F52A,2.28,2.28,4,2022-12-31
...,...,...,...,...,...,...
87,354013,L63B,1.07,1.07,4,2022-12-31
88,354186,G18B,1.71,1.71,3,2022-12-31
89,354882,G46A,1.85,1.85,4,2022-12-31
90,355117,G08A,2.09,2.09,4,2022-12-31


In [10]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 09:11:35.646 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 92 cases into the 'Revisions' table ...
2022-11-21 09:11:41.011 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 92 cases into the 'Revisions' table
2022-11-21 09:11:41.016 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 877 rows into the 'Diagnoses' table ...
2022-11-21 09:12:02.769 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 877 rows into the 'Diagnoses' table
2022-11-21 09:12:02.774 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 568 rows into the 'Procedures' table ...
2022-11-21 09:12:06.746 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 568 rows into the 'Procedures' table
